### Optimization Data Additional Train

In [36]:
import pandas as pd

df_original = pd.read_csv('final-6.csv')
df1 = pd.read_csv('sensor_data_Exercise_expanded.csv', index_col=False)
df2 = pd.read_csv('sensor_data_Sleep.csv', index_col=False)
df3 = pd.read_csv('sensor_data_study.csv', index_col=False)
df4 = pd.read_csv('sensor_data_study2.csv', index_col=False)

In [38]:
df1 = df1.drop(columns=['sec','week','idx1','idx2','idx3','idx4','idx5','idx6','idx7','idx8','idx9','idx0'])
df2 = df2.drop(columns=['sec','week','idx1','idx2','idx3','idx4','idx5','idx6','idx7','idx8','idx9','idx0'])
df3 = df3.drop(columns=['sec','week','idx1','idx2','idx3','idx4','idx5','idx6','idx7','idx8','idx9','idx0'])
df4 = df4.drop(columns=['sec','week','idx1','idx2','idx3','idx4','idx5','idx6','idx7','idx8','idx9','idx0'])

In [47]:
df_final = pd.concat([df_original, df1, df2, df3, df4], axis=0)

In [50]:
df_final = df_final[df_final['peak']!=0]

In [52]:
sleep_df = df_final[df_final['category'] == 'sleep'].sample(frac=1/3, random_state=42)
not_else_df = df_final[df_final['category'] != 'else']
df_final = pd.concat([not_else_df[not_else_df['category'] != 'sleep'], sleep_df], ignore_index=True)

In [104]:
df_final['category'].value_counts()

,count
category,
sleep,3646
study,2643
class,2167
exercise,870


### Start LSTM model

In [54]:
# 1️⃣ LSTM 모델 구성 코드
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [105]:
# Load and preprocess data
#df = pd.read_csv('final.csv')
df = df_final

encoder = LabelEncoder()
Y = encoder.fit_transform(df['category'])
X = df.drop(columns=['category'])

In [106]:
print(encoder.classes_)

['class' 'exercise' 'sleep' 'study']


In [107]:
len(X)

9326

In [108]:
# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [109]:
print(scaler.mean_)
print(scaler.scale_)

np.save("scaler_mean.npy", scaler.mean_)
np.save("scaler_std.npy", scaler.scale_)

print(len(scaler.mean_))
print(len(scaler.scale_))

[ 3.55711037e+01  1.29210991e+02  3.93609050e+02  1.54503800e-02
  6.40686302e-01  8.37711789e+00 -1.11793005e-02  4.03758740e-03
  6.46861238e-03  3.47847913e+01  5.39033026e+02  2.85467458e+04
  1.50768445e+04  6.95140192e+03  3.08457078e+03  2.06106495e+03
  1.50435422e+03  1.11571127e+03  8.78133975e+02  7.69793023e+02
  9.11730350e+02  4.27674244e+01 -5.77403174e+02 -4.27532704e+01
 -4.49723354e+01 -4.64459575e+01 -5.10219816e+01 -6.09737294e+01
 -6.39369505e+01 -6.52191722e+01 -6.65794553e+01 -6.82439417e+01
 -6.94638645e+01  2.32361141e-01]
[2.70220400e-03 2.77953032e-02 5.58068807e+02 2.46795624e+00
 2.46804635e+00 4.32149168e+00 4.70540796e-01 4.35744194e-01
 5.51583866e-01 1.21240454e+01 1.47703994e+03 9.34973726e+04
 4.80093097e+04 2.26681368e+04 9.44236065e+03 7.07635565e+03
 5.01721435e+03 2.96301612e+03 2.23216576e+03 1.81452642e+03
 1.90956045e+03 2.81920981e+01 8.12881152e+01 1.16196111e+01
 1.07045643e+01 1.01585979e+01 7.50565227e+00 1.12457992e+01
 1.31429190e+01 1.4

In [110]:
# Reshape to (samples, timesteps=12, features)
X_seq = []
y_seq = []
seq_length = 12

previous_label = Y[0]

i = 0
while i <= len(X) - seq_length:
    current_label = Y[i + seq_length - 1]

    if current_label != previous_label:
        previous_label = Y[i + 1]
        i += 1
    else:
        X_seq.append(X[i:i+seq_length])
        y_seq.append(current_label)
        previous_label = current_label
        i += seq_length

X_seq = np.array(X_seq)
y_seq = to_categorical(y_seq)

In [111]:
X_seq.shape

(774, 12, 34)

In [112]:
y_seq.shape

(774, 4)

In [113]:
y_labels = np.argmax(y_seq, axis=1)

# 빈 리스트로 초기화
X_train_list, X_test_list = [], []
y_train_list, y_test_list = [], []

unique_labels = np.unique(y_labels)

# 각 활동별로 마지막 10% 데이터만 뽑아서 테스트셋으로 만들기.
for label in unique_labels:
    idx = np.where(y_labels == label)[0]
    count = len(idx)
    split_idx = int(count * 0.9)

    train_idx = idx[:split_idx]
    test_idx = idx[split_idx:]

    X_train_list.append(X_seq[train_idx])
    y_train_list.append(y_seq[train_idx])

    X_test_list.append(X_seq[test_idx])
    y_test_list.append(y_seq[test_idx])

# 리스트를 다시 하나의 배열로 합치기
X_train = np.concatenate(X_train_list)
y_train = np.concatenate(y_train_list)
X_test = np.concatenate(X_test_list)
y_test = np.concatenate(y_test_list)

In [114]:
# Build model
model = Sequential()
model.add(LSTM(64, input_shape=(seq_length, X_seq.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_seq.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [115]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
model.save("activity_model.h5")

Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.7354 - loss: 0.9342 - val_accuracy: 0.9747 - val_loss: 0.1802
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9914 - loss: 0.1427 - val_accuracy: 1.0000 - val_loss: 0.0374
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9970 - loss: 0.0340 - val_accuracy: 1.0000 - val_loss: 0.0120
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9993 - loss: 0.0120 - val_accuracy: 1.0000 - val_loss: 0.0062
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.0071 - val_accuracy: 1.0000 - val_loss: 0.0040
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 1.0000 - loss: 0.0038 - val_accuracy: 1.0000 - val_loss: 0.0028
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 1.0000 - loss: 0.0028 - val_accuracy: 1.0000 - val_loss: 0.0021
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 1.0000 - v

### Tensorflow Lite Convert

In [116]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.experimental_enable_resource_variables = True

converter._experimental_lower_tensor_list_ops = False

converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,     # 기본 TFLite 연산
    tf.lite.OpsSet.SELECT_TF_OPS        # TF 연산 일부 직접 허용
]

tflite_model = converter.convert()

with open("activity_model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmps7344x9b'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 12, 34), dtype=tf.float32, name='keras_tensor_14')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  137451219048336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137451219055632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137451219055440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137451219049488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137451219054864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137451219047568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137451219055248: TensorSpec(shape=(), dtype=tf.resource, name=None)


### Original Model Inference

새로운 데이터셋으로 테스트하는 경우

In [ ]:
df_test = pd.read_csv('sensor_data_test_study2.csv')

In [ ]:
X = X.drop(columns=['sec','week','idx1','idx2','idx3','idx4','idx5','idx6','idx7','idx8','idx9','idx0'])

In [ ]:
X = scaler.transform(X)

In [ ]:
X_seq = []
seq_length = 12

for i in range(0, len(X) - seq_length, seq_length):
    X_seq.append(X[i:i+seq_length])

In [ ]:
X_seq = np.array(X_seq)
X_seq.shape

(3, 12, 34)

기존 수집된 데이터의 일부를 사용하는 경우 (학습에 사용 안한)

In [117]:
X_test.shape

(79, 12, 34)

In [118]:
predictions = model.predict(X_test)

print(predictions)

# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Decode class labels back to original categories
predicted_categories = encoder.inverse_transform(predicted_classes)

print("Predicted categories for X_seq:")
predicted_categories

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
[[9.99821186e-01 5.06584656e-05 1.04908177e-05 1.17719996e-04]
 [9.99838591e-01 4.85149831e-05 9.37551522e-06 1.03628605e-04]
 [9.99813557e-01 5.49782308e-05 1.13335582e-05 1.20274897e-04]
 [9.99804735e-01 5.39891953e-05 1.15343082e-05 1.29775828e-04]
 [9.99800205e-01 5.32328377e-05 1.18339149e-05 1.34781934e-04]
 [9.99819040e-01 5.12180741e-05 1.06633925e-05 1.19079225e-04]
 [9.99805927e-01 5.30048783e-05 1.15480798e-05 1.29546606e-04]
 [9.99805212e-01 5.24314491e-05 1.15702514e-05 1.30862856e-04]
 [9.99795675e-01 5.30943871e-05 1.18462813e-05 1.39543845e-04]
 [9.99797821e-01 5.30735451e-05 1.18815396e-05 1.37280120e-04]
 [9.99802351e-01 5.31372934e-05 1.15297280e-05 1.33002890e-04]
 [9.99802351e-01 5.43162787e-05 1.16609090e-05 1.31541150e-04]
 [9.99794722e-01 6.03274530e-05 1.30400840e-05 1.32035711e-04]
 [9.99806166e-01 5.77519240e-05 1.20763925e-05 1.24082813e-04]
 [9.99793291e-01 5.63222602e-05 1.25606175e-05 1.37792100e-04]
 [9.99809027e-01

array(['class', 'class', 'class', 'class', 'class', 'class', 'class',
       'class', 'class', 'class', 'class', 'class', 'class', 'class',
       'class', 'class', 'class', 'class', 'exercise', 'exercise',
       'exercise', 'exercise', 'exercise', 'exercise', 'exercise',
       'exercise', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'study', 'study', 'study',
       'study', 'study', 'study', 'study', 'study', 'study', 'study',
       'study', 'study', 'study', 'study', 'study', 'study', 'study',
       'study', 'study', 'study', 'study', 'study'], dtype=object)

In [119]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

true_classes = np.argmax(y_test, axis=1)

# 디코딩
predicted_labels = encoder.inverse_transform(predicted_classes)
true_labels = encoder.inverse_transform(true_classes)

# 혼동 행렬
conf_mat = confusion_matrix(true_labels, predicted_labels, labels=encoder.classes_)

# Pandas DataFrame으로 보기 좋게 출력
conf_df = pd.DataFrame(conf_mat, index=encoder.classes_, columns=encoder.classes_)

print("🔢 혼동 행렬 (Confusion Matrix):")
print(conf_df)

# 분류 리포트 (정밀도, 재현율, F1 등 포함)
print("\n📊 분류 성능 보고서:")
print(classification_report(true_labels, predicted_labels, target_names=encoder.classes_))

🔢 혼동 행렬 (Confusion Matrix):
          class  exercise  sleep  study
class        18         0      0      0
exercise      0         8      0      0
sleep         0         0     31      0
study         0         0      0     22

📊 분류 성능 보고서:
              precision    recall  f1-score   support

       class       1.00      1.00      1.00        18
    exercise       1.00      1.00      1.00         8
       sleep       1.00      1.00      1.00        31
       study       1.00      1.00      1.00        22

    accuracy                           1.00        79
   macro avg       1.00      1.00      1.00        79
weighted avg       1.00      1.00      1.00        79



In [120]:
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)  # y_test가 one-hot인 경우

# 각 예측에서 softmax 최대값 (confidence)
confidences = np.max(predictions, axis=1)

# 예측된 클래스별 평균 confidence 계산
results = {}
for cls in np.unique(predicted_classes):
    class_indices = np.where(predicted_classes == cls)[0]
    class_confidences = confidences[class_indices]
    results[cls] = {
        'count': len(class_confidences),
        'mean_confidence': np.mean(class_confidences)
    }

# 보기 좋게 출력
df = pd.DataFrame(results).T
df.index.name = 'Predicted_Class'
df.columns = ['Sample_Count', 'Mean_Confidence']
df = df.sort_index()
print(df)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
                 Sample_Count  Mean_Confidence
Predicted_Class                               
0                        18.0         0.999804
1                         8.0         0.998813
2                        31.0         0.999705
3                        22.0         0.999834


### Mobile model Inference

In [121]:
interpreter = tf.lite.Interpreter(model_path="activity_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

X_test = X_test.astype(np.float32)  # TFLite는 float32 요구

# 5. 예측 루프
predicted_categories = []

for i in range(X_test.shape[0]):
    input_tensor = X_test[i:i+1]  # shape: (1, 12, 35)

    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])  # shape: (1, num_classes)
    print(output)

    predicted_index = np.argmax(output[0])  # 예측된 클래스 인덱스
    predicted_label = encoder.inverse_transform([predicted_index])[0]  # 실제 라벨명으로 변환
    predicted_categories.append(predicted_label)


print("Predicted categories for X_seq:")
print(predicted_categories)

[[9.9982113e-01 5.0658458e-05 1.0490808e-05 1.1771988e-04]]
[[9.99838471e-01 4.85150231e-05 9.37551431e-06 1.03628576e-04]]
[[9.9981350e-01 5.4978173e-05 1.1333558e-05 1.2027477e-04]]
[[9.9980468e-01 5.3989184e-05 1.1534307e-05 1.2977568e-04]]
[[9.9980026e-01 5.3232889e-05 1.1833927e-05 1.3478180e-04]]
[[9.99819100e-01 5.12180195e-05 1.06633925e-05 1.19079115e-04]]
[[9.9980587e-01 5.3004867e-05 1.1548081e-05 1.2954658e-04]]
[[9.9980515e-01 5.2431438e-05 1.1570251e-05 1.3086283e-04]]
[[9.9979562e-01 5.3094380e-05 1.1846269e-05 1.3954383e-04]]
[[9.9979776e-01 5.3073534e-05 1.1881538e-05 1.3727997e-04]]
[[9.9980229e-01 5.3137235e-05 1.1529727e-05 1.3300263e-04]]
[[9.9980253e-01 5.4316231e-05 1.1660901e-05 1.3154116e-04]]
[[9.99794662e-01 6.03274493e-05 1.30400695e-05 1.32035682e-04]]
[[9.9980611e-01 5.7751920e-05 1.2076391e-05 1.2408278e-04]]
[[9.9979335e-01 5.6322206e-05 1.2560604e-05 1.3779195e-04]]
[[9.9980885e-01 5.6788598e-05 1.1636512e-05 1.2269730e-04]]
[[9.9982184e-01 5.6348901e-0

In [122]:
true_classes = np.argmax(y_test, axis=1)

# 디코딩
predicted_labels = encoder.inverse_transform(predicted_classes)
true_labels = encoder.inverse_transform(true_classes)

# 혼동 행렬
conf_mat = confusion_matrix(true_labels, predicted_labels, labels=encoder.classes_)

# Pandas DataFrame으로 보기 좋게 출력
conf_df = pd.DataFrame(conf_mat, index=encoder.classes_, columns=encoder.classes_)

print("🔢 혼동 행렬 (Confusion Matrix):")
print(conf_df)

# 분류 리포트 (정밀도, 재현율, F1 등 포함)
print("\n📊 분류 성능 보고서:")
print(classification_report(true_labels, predicted_labels, target_names=encoder.classes_))

🔢 혼동 행렬 (Confusion Matrix):
          class  exercise  sleep  study
class        18         0      0      0
exercise      0         8      0      0
sleep         0         0     31      0
study         0         0      0     22

📊 분류 성능 보고서:
              precision    recall  f1-score   support

       class       1.00      1.00      1.00        18
    exercise       1.00      1.00      1.00         8
       sleep       1.00      1.00      1.00        31
       study       1.00      1.00      1.00        22

    accuracy                           1.00        79
   macro avg       1.00      1.00      1.00        79
weighted avg       1.00      1.00      1.00        79



In [123]:
confidences = np.max(predictions, axis=1)

results = {}
for cls in np.unique(predicted_classes):
    class_indices = np.where(predicted_classes == cls)[0]
    class_confidences = confidences[class_indices]
    results[cls] = {
        'count': len(class_confidences),
        'mean_confidence': np.mean(class_confidences)
    }

# 보기 좋게 출력
df = pd.DataFrame(results).T
df.index.name = 'Predicted_Class'
df.columns = ['Sample_Count', 'Mean_Confidence']
df = df.sort_index()
print(df)

                 Sample_Count  Mean_Confidence
Predicted_Class                               
0                        18.0         0.999804
1                         8.0         0.998813
2                        31.0         0.999705
3                        22.0         0.999834
